# Подключение необходимых для работы библиотек

In [1]:
import numpy as np
import scipy as sp
from scipy.linalg import lu
import copy

# Выражение для исходной системы AX = f

In [2]:
A = np.array([[1, 0, 0, 0, 0, 0, 0, 0, 0],
             [1, -2, 1, 0, 0, 0, 0, 0, 0],
             [0, 1, -2, 1, 0, 0, 0, 0, 0],
             [0, 0, 1, -2, 1, 0, 0, 0, 0],
             [0, 0, 0, 1, -2, 1, 0, 0, 0],
             [0, 0, 0, 0, 1, -2, 1, 0, 0],
             [0, 0, 0, 0, 0, 1, -2, 1, 0],
             [0, 0, 0, 0, 0, 0, 1, -2, 1],
             [1, 2, 2, 2, 2, 2, 2, 2, 1]])

f = np.array([1, 2 / 4, 2 / 9, 2 / 16, 2 / 25, 2 / 36, 2 / 49, 2 / 64, -3])


# Вспомогательные функции

In [3]:
def norm(x):
    abs_arr = np.absolute(x)
    return np.amax(abs_arr)

# Метод Зейделя

In [4]:
def seidel_method(A, f, precision):
    
    D = np.diag(np.diag(A))
    U = np.triu(A, 1)
    L = np.tril(A, -1)
    
    inverted = np.linalg.inv(D + L)
    
    B = -1 * inverted.dot(U)
    print(inverted)
    print(f)
    F = inverted.dot(f)
    
    n = f.size
    x = np.zeros(n)
    
    iters = 0
    
    error = norm(A.dot(x) - f)
    while (error > precision):
        x = B.dot(x) + F
        error = norm(A.dot(x) - f)
        
        print('Невязка на {0} итерации: '.format(iters))
        print(error)
        
        print('Решение на {0} итерации: '.format(iters))
        print(x)
        print('\n\n\n')
        
        iters += 1
        
    return x, error


# Метод Гаусса

In [5]:
def element_swap(A):
    n = A[0].size
    max_elem = 0
    index = 0
    
    for i in range(n):
        if (A[i][0] > max_elem):
            max_elem = A[i][0]
            index = i
                
    A[[0, index]] = A[[index, 0]]
    return A

test_A = np.array([[4, 1, 1],
                  [3, 5, 1],
                  [1, 1, 3]], dtype=float)

test_F = np.array([3, 7, 4], dtype=float)


def gauss_direct(A, f):
    A = element_swap(A)
    n = A[0].size
    if (n == 1):
        return
    
    for i in range(1, n):
        factor = A[i, 0] / A[0, 0]
        A[i] = A[i] - factor * A[0]
        f[i] = f[i] - factor * f[0]
        
    gauss_direct(A[1:, 1:], f[1:])
    
    return A, f

def gauss_reverse(A, f):
    
    n = A[0].size
    if (n == 1):
        return
    
    for i in reversed(range(0, n - 1)):
        factor = A[i, n - 1] / A[n - 1, n - 1]
        A[i] = A[i] - factor * A[n - 1]
        f[i] = f[i] - factor * f[n - 1]
    
    gauss_reverse(A[:(n - 1), :(n - 1)], f[:(n - 1)])
    
    ans = np.zeros(n)
    for i in range(n):
        ans[i] = f[i] / A[i][i]
    
    return ans

# Метод Якоби

In [6]:
ITERATION_LIMIT = 1000
SIGMA = 1e-8
all_errors_list = []

def getErrorMatrixNorm(errors):
    return np.sqrt(sum(errors[ind]**2 for ind in range(len(errors))) )

def m_getErrorMatrixNorm(A, x, b):
    error = np.dot(A, x) - b
    #print(f'error value:{error}')
    return getErrorMatrixNorm(error.tolist())

def jakobi(A, f):
    x = np.zeros_like(f)
    for it_count in range(ITERATION_LIMIT):
        if it_count != 0:
            print(f"Iteration {it_count}: {x}")

        all_errors_list.append(m_getErrorMatrixNorm(A, x, f))

        x_new = np.zeros_like(x)

        for i in range(A.shape[0]):
            s1 = np.dot(A[i, :i], x[:i])
            s2 = np.dot(A[i, i + 1:], x[i + 1:])
            x_new[i] = (f[i] - s1 - s2) / A[i, i]
            if x_new[i] == x_new[i-1]:
                break

        #if np.allclose(x, x_new, atol=1e-8, rtol=0.):
        if m_getErrorMatrixNorm(A, x, f) <= SIGMA:
            break

        x = x_new
    return x

# Метод LU

In [7]:

def _initValue_(n=99):
    return [ ind+1 for ind in range( n+1 )]

def lu(A, f):
    return
    L, U = lu(A, f)
    LU = np.matmul(L, U)

    if np.allclose(A, LU):
        print('LU decomposition is valid.')
        print('Continue...')

        y = np.matmul(np.linalg.inv(L), f)

        x = np.matmul(np.linalg.inv(U), y)
        print('x=', x.tolist())
    else:
        print('LU decomposition is not valid.')

A_list = A.tolist()
value_list = _initValue_()


# Метод верхней релаксации

In [8]:
def sor_solver(A, b, omega, initial_guess, convergence_criteria):
    """
    This is an implementation of the pseudo-code provided in the Wikipedia article.
    Arguments:
        A: nxn numpy matrix.
        b: n dimensional numpy vector.
        omega: relaxation factor.
        initial_guess: An initial solution guess for the solver to start with.
        convergence_criteria: The maximum discrepancy acceptable to regard the current solution as fitting.
    Returns:
        phi: solution vector of dimension n.
    """
    step = 0
    phi = initial_guess[:]
    
    residual = m_getErrorMatrixNorm(A, initial_guess, b)#linalg.norm(A @ phi - b)  # Initial residual
    
    all_errors_list = []
    
    while residual > convergence_criteria:
        
        all_errors_list.append(m_getErrorMatrixNorm(A, phi, b))

        for i in range(A.shape[0]):
            sigma = 0
            for j in range(A.shape[1]):
                if j != i:
                    sigma += A[i, j] * phi[j]
            phi[i] = (1 - omega) * phi[i] + (omega / A[i, i]) * (b[i] - sigma)
        residual =np.linalg.norm(A @ phi - b)
        step += 1
        print("Step {} Residual: {:10.6g}".format(step, residual))
    
    return phi

# An example case that mirrors the one in the Wikipedia article
omega = 0.5  # Relaxation factor


initial_guess = np.zeros_like(f, np.float_)

phi = sor_solver(A, f, omega, initial_guess, SIGMA)


Step 1 Residual:    2.30263
Step 2 Residual:      1.404
Step 3 Residual:   0.797239
Step 4 Residual:   0.710241
Step 5 Residual:    0.72386
Step 6 Residual:   0.650236
Step 7 Residual:   0.527601
Step 8 Residual:   0.406556
Step 9 Residual:   0.310619
Step 10 Residual:   0.242512
Step 11 Residual:   0.196296
Step 12 Residual:   0.164829
Step 13 Residual:   0.142268
Step 14 Residual:    0.12454
Step 15 Residual:   0.109276
Step 16 Residual:  0.0954235
Step 17 Residual:  0.0826948
Step 18 Residual:  0.0711073
Step 19 Residual:  0.0607248
Step 20 Residual:  0.0515592
Step 21 Residual:  0.0435607
Step 22 Residual:  0.0366389
Step 23 Residual:  0.0306858
Step 24 Residual:  0.0255913
Step 25 Residual:  0.0212513
Step 26 Residual:  0.0175708
Step 27 Residual:  0.0144642
Step 28 Residual:   0.011855
Step 29 Residual: 0.00967498
Step 30 Residual:  0.0078632
Step 31 Residual: 0.00636579
Step 32 Residual: 0.00513525
Step 33 Residual:    0.00413
Step 34 Residual: 0.00331387
Step 35 Residual: 0.002

# Непосредственно программа с тестированием методов

In [10]:
res, error = seidel_method(test_A.copy(), test_F.copy(), 10e-14)
print('Итоговое решение по Зейделю: ')
print(res)
print('Итоговая невязка: ')
print(error)
print('\n\n\n')

resA, resf = gauss_direct(test_A.copy(), test_F.copy())
res = gauss_reverse(resA.copy(), resf.copy())
print('Итоговое решение по Гауссу:')
print(res)
print('\n\n\n')

res = jakobi(test_A.copy(), test_F.copy())
print('Итоговое решение по Якоби:')
print(res)
print('\n\n\n')


lu(test_A.copy(), test_F.copy())
print('Итоговое решение по LU разложению:')
print(res)
print('\n\n\n')
omega = 0.5  # Relaxation factor


# An example case that mirrors the one in the Wikipedia article
omega = 0.5  # Relaxation factor


initial_guess = np.zeros_like(f, np.float_)

phi = sor_solver(A, f, omega, initial_guess, SIGMA)
print('Итоговое решение по методу верхней релаксации:')
print(res)
print('\n\n\n')



print('Правильное решение через питоновские алгоритмы (для проверки): ')
res = np.linalg.solve(test_A.copy(), test_F.copy())
print(res)
print('\n\n\n')



[[ 0.25        0.          0.        ]
 [-0.15        0.2         0.        ]
 [-0.03333333 -0.06666667  0.33333333]]
[3. 7. 4.]
Невязка на 0 итерации: 
1.7166666666666668
Решение на 0 итерации: 
[0.75       0.95       0.76666667]




Невязка на 1 итерации: 
0.2124999999999999
Решение на 1 итерации: 
[0.32083333 1.05416667 0.875     ]




Невязка на 2 итерации: 
0.024513888888888946
Решение на 2 итерации: 
[0.26770833 1.064375   0.88930556]




Невязка на 3 итерации: 
0.0025868055555555713
Решение на 3 итерации: 
[0.26157986 1.06519097 0.89107639]




Невязка на 4 итерации: 
0.0002381365740742325
Решение на 4 итерации: 
[0.26093316 1.06522483 0.89128067]




Невязка на 5 итерации: 
2.1556712963999303e-05
Решение на 5 итерации: 
[0.26087363 1.06521969 0.89130223]




Невязка на 6 итерации: 
1.984471451166314e-06
Решение на 6 итерации: 
[0.26086952 1.06521784 0.89130421]




Невязка на 7 итерации: 
2.396195024623182e-07
Решение на 7 итерации: 
[0.26086949 1.06521747 0.89130435]




Невяз

# Подсчет корней характеристического уравнения и числа обусловленности

In [11]:
lambdas, eigenvectors = np.linalg.eig(A)
print(lambdas)

lambda_max = lambdas.max()
lambda_min = lambdas.min()

print('Максимальный корень ХУ: ')
print(lambda_max)
print('Минимальный корень ХУ: ')
print(lambda_min)

print('\n\n\n')

norm_a = norm(A)
norm_a_inversed = norm(np.linalg.inv(A))

print(norm_a)
print(norm_a_inversed)

print('Число обусловленности: ')
print(norm_a * norm_a_inversed)

[ 1.79620635 -3.8571539  -3.41421356 -2.86417214 -2.         -1.55703634
 -0.51784397 -0.58578644  1.        ]
Максимальный корень ХУ: 
1.7962063457382964
Минимальный корень ХУ: 
-3.8571538967671066




2
1.9999999999999996
Число обусловленности: 
3.999999999999999


![Текст с описанием картинки](jacobi.jpg)

![Текст с описанием картинки](seidel.jpg)

![Текст с описанием картинки](relaxation.jpg)